# AGAVE
A Graphical Abstract Verification Engine

In [1]:
%load_ext autoreload
%autoreload 2

## Setup

In [2]:
from agave.engine.data_access_layer import CooccurrencyGraph, PaperCache, Metadata

In [3]:
from agave.engine.query_manager import MichaelScott as Manager

In [4]:
graph_db_uri = 'neo4j+s://91bd4f79.databases.neo4j.io'
graph_db_user = 'neo4j'
graph_db_password = 'syc9_tjj2ECTW03hPNtfSN4j6G6Q6lKUUp7YkYP3ALc'
graph_db = CooccurrencyGraph(graph_db_uri, graph_db_user, graph_db_password)

In [5]:
graph_db.test_connection()

Connection established


In [6]:
paper_cache = PaperCache('~/Documents/tesi_local/agave-0/entity_paper.db', '~/Documents/tesi_local/agave-0/bigram_paper.db')

In [101]:
metadata = Metadata('~/Code/CovEffect-data-provision/products/raw/metadata.csv', sample=True)

## Querying

#### Note

- Ho una chain di entities
- paths per ogni coppia di entities
- Per ogni coppia:
    - scelgo un path, prendo i papers
    - metto i papers nel gatherer, mi segno le relazioni, mi segno se contengono quelle della query
- Ordino i paper:
    - Prima quelli che spiegano più della catena originale
    - Secondi quelli che spiegano tanti nodi comunque
    - Gli altri per data e qualità
    

#### 

In [152]:
the_manager = Manager(metadata, paper_cache, graph_db)

In [153]:
chain = ['BRCA1','paracetamol','death','breast cancer']
the_manager.find_single_chain(chain)

In [154]:
the_manager.show_single_chain_result()

BRCA1-->paracetamol
0 ['BRCA1<SEP>patients', 'paracetamol<SEP>patients'] 			 51.5
-------------------------
paracetamol-->death
0 ['paracetamol<SEP>patients', 'death<SEP>patients'] 			 5416.0
1 ['COVID-19<SEP>paracetamol', 'COVID-19<SEP>death'] 			 5141.0
2 ['paracetamol<SEP>patient', 'death<SEP>patient'] 			 1493.0
3 ['drug<SEP>paracetamol', 'death<SEP>drug'] 			 479.5
4 ['children<SEP>paracetamol', 'children<SEP>death'] 			 463.0
5 ['fever<SEP>paracetamol', 'death<SEP>fever'] 			 445.5
6 ['pain<SEP>paracetamol', 'death<SEP>pain'] 			 108.0
-------------------------
death-->breast cancer
0 ['breast cancer<SEP>death'] 			 103.0
-------------------------


In [155]:
the_manager.select_path_single_chain([0,2,0])

BRCA1-->paracetamol
0 ['BRCA1<SEP>patients', 'paracetamol<SEP>patients']
death-->breast cancer
2 ['paracetamol<SEP>patient', 'death<SEP>patient']
BRCA1-->paracetamol
0 ['breast cancer<SEP>death']


In [156]:
#the_manager.stoner.get_selected_path_relations()

In [157]:
the_manager.gather_papers()

In [158]:
the_manager.gatherer.papers

,cord_uid,relation,original
0,l0wc31to,BRCA1<SEP>patients,True
1,ikstn4yk,BRCA1<SEP>patients,True
2,bhsw2wv4,BRCA1<SEP>patients,True
3,57i1jaol,BRCA1<SEP>patients,True
4,0qewmjn1,BRCA1<SEP>patients,True
...,...,...,...
3182,35z11126,breast cancer<SEP>death,True
3183,817f3rer,breast cancer<SEP>death,True
3184,oduk7qax,breast cancer<SEP>death,True
3185,398je5tr,breast cancer<SEP>death,True


In [174]:
the_manager.gatherer.extract_papers()

In [176]:
the_manager.gatherer.extracted_papers.head()

,cord_uid,title,abstract,doi,authors,journal,publish_time,occurrences,explained_relations
0,00rezmqj,Intra-arterial administration of a replication...,Both replication-incompetent and replication-s...,10.1038/sj.gt.3301512,"Reid, T; Galanis, E; Abbruzzese, J; Sze, D; An...",Gene Ther,2001-11-22 00:00:00,1,['death<SEP>patient']
1,0nyy7vjo,Tuberculosis mortality: patient characteristic...,"BACKGROUND: In the antibiotic era, tuberculosi...",10.1186/1471-2334-14-5,"Lin, Chou-Han; Lin, Chou-Jui; Kuo, Yao-Wen; Wa...",BMC Infect Dis,2014-01-03 00:00:00,1,['death<SEP>patient']
2,18dxcn41,Demographic profile and outcome analysis of a ...,Objective : To study the profile and outcome o...,10.1007/bf02724117,"Khilnani, Praveen; Sarma, Devajit; Singh, Reet...",Indian J Pediatr,2004-01-01 00:00:00,1,['death<SEP>patient']
3,1vimqhdp,36th International Symposium on Intensive Care...,P001 - Sepsis impairs the capillary response w...,10.1186/s13054-016-1208-6,"Bateman, R. M.; Sharpe, M. D.; Jagger, J. E.; ...",Crit Care,2016-04-20 00:00:00,1,['death<SEP>patient']
4,271vldil,Understanding the clinical spectrum of complic...,The resurgence of the malaria eradication agen...,10.1186/1475-2875-11-12,"Lacerda, Marcus VG; Mourão, Maria PG; Alexandr...",Malar J,2012-01-09 00:00:00,1,['death<SEP>patient']
